In [ ]:
# import all libraries
from scipy.stats import sem as sem
import sys
import glob

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

normal analysis

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_combined.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

if os.path.exists(f"{main_dir}/outputs_extracted/results"):
    results_folder = f"{main_dir}/outputs_extracted/results"
elif os.path.exists(f"{main_dir}/outputs/results"):
    results_folder = f"{main_dir}/outputs/results"
else:
    raise ValueError(
        f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline"
    )

output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

In [ ]:
all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    net_file=net_file,
    output_folder=output_folder,
    analysis_ext=ana_file,
)

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False, use_cinnabar=False)

is uncertainty for a single run the same across triplicates?
does it agree with uncertainty across triplicates?

In [ ]:
# error for a perturbation for triplicate

uncertainty_dict = {}

for eng in all_analysis_object.engines:
    uncertainty_dict[eng] = {}
    print(f"{eng}")

    for pert in all_analysis_object.calc_pert_dict[eng]:
        uncertainty_dict[eng][pert] = all_analysis_object.calc_pert_dict[eng][pert][1]

# uncertainty_dict

In [ ]:
# error for a perturbation per single run

uncertainty_dict_single = {}

for eng in all_analysis_object.engines:
    uncertainty_dict_single[eng] = {}
    repeat = 0
    for file in all_analysis_object._results_repeat_files[eng]:
        uncertainty_dict_single[eng][repeat] = {}
        calc_diff_dict = make_dict.comp_results(
            file, all_analysis_object.perturbations, eng, name=None
        )

        for pert in calc_diff_dict.keys():
            uncertainty_dict_single[eng][repeat][pert] = calc_diff_dict[pert][1]

        repeat += 1

make list of single run uncertainties
can feed into histogram function and also average to compare

abstract histogram plotting so can put in data
abstract other plotting functions so can put in whatever
also need to abstract colour?

In [ ]:
# histogram
all_analysis_object.plot_histogram_sem(pert_val="pert")

all_analysis_object.plot_histogram_runs()

Comparing the triplicate result to the single result compared to experimental

In [ ]:
# uncertainty for triplicates compared to experimental

for eng in all_analysis_object.engines:
    eng_mue = all_analysis_object._stats_object.compute_mue("pert", y=eng)
    print(f"MUE for {eng} is {eng_mue}")

In [ ]:
# uncetainty for single runs compared to experimental

for eng in all_analysis_object.engines:
    repeat = 0

    for file in all_analysis_object._results_repeat_files[eng]:
        calc_diff_dict = make_dict.comp_results(
            file, all_analysis_object.perturbations, eng, name=None
        )

        df = plotting_engines.match_dicts_to_df(
            all_analysis_object.exper_pert_dict, calc_diff_dict, "experimental", "calc"
        )

        df = df.dropna()

        x = df[f"freenrg_experimental"]
        y = df[f"freenrg_calc"]
        xerr = df[f"err_experimental"]
        yerr = df[f"err_calc"]

        res = stats_engines.compute_stats(x, y, xerr, yerr, "MUE")

        print(f"MUE for {eng} repeat {repeat} is {res}")
        repeat += 1